## iOS 앱의 구조와 코코아 터치 프레임워크
## 앱의 기본 구조
## 엔트리 포인트와 앱의 초기화 과정

### C 언어에 뿌리를 둔 모든 애플리케이션은 main() 함수로부터 시작된다
- #### 이를 "엔트리 포인트 (Entry Point, 시작 진입점)"라고 한다
- #### 운영체제가 애플리케이션 내부에 정의된 main() 함수를 찾아 호출하면 여기에 작성된 코드들이 연쇄적으로 실행되면서 우리가 작성해 둔 커스텀 코드에까지 도달하게 되는 식이다

### Objective-C 역시 C언어에 기반하고 있기 때문에,
- #### 이를 이용하여 만들어진 iOS 앱도 main() 함수로부터 시작된다

### C 기반의 다른 애플리케이션과 차이가 있다면, 
- #### iOS 앱에서는 main() 함수를 우리가 직접 작성하지 않는다는 것 정도이다

### 대신 Xcode 프로젝트를 생성하면 main() 함수가 자동으로 만들어지는데,
- #### 여기에는 iOS 앱이 실행될 때 처리해야 할 내용이 작성되어 있기 때문에 main() 함수를 전혀 건드릴 필요가 없다

### 실제로 Objective-C 기반의 Xcode 프로젝트를 생성했을 때 main.m 파일 안에 생성되는 main() 함수

```
#import <UIKit/UIKit.h>
#import "AppDelegate.h"

int main(int argc, char * argv[]) {
    @autoreleasepool {
        return UIApplicationMain(argc, argv, nil, NSStringFromClass([AppDelegate class]));
    }
}
```

### main() 함수가 하는 일은 단순하다
- #### 실행 시 시스템으로부터 전달받은 두 개의 인자값과 AppDelegate 클래스를 이용하여 UIApplicationMain() 함수를 호출
    - #### 그 결과로 UIApplication 객체를 반환한다
        - #### 생성된 UIApplication 객체는 UIKit 프레임워크에 속해 있으므로 이후의 앱 제어권은 UIKit 프레임워크로 이관된다

### main() 함수가 C 기반 애플리케이션의 엔트리 포인트라면, UIApplicationMain() 함수는 그 중에서도 iOS 앱에 속하는 부분의 엔트리 포인트라고 할 수 있다
- #### 이 함수는 앱의 핵심 객체를 생성하는 프로세스를 핸들링하고, 스토리보드 파일로부터 앱의 유저 인터페이스를 읽어 들일뿐만 아니라 우리가 작성한 커스텀 코드를 호출해 줌으로써 앱 생성 초기에 필요한 설정을 구현할 수 있게 해준다
    - #### 여기에 더해서 이벤트를 입력받기 위한 이벤트 루프를 실행시키기도 한다
        - #### 이 과정에서 우리가 직접 구현해야 하는 것은 스토리보드 파일이나 초기화를 위한 커스텀 코드뿐이다

### UIApplicationMain() 함수가 생성하는 UIApplication은 앱의 본체라고 할 수 있는 객체
- #### 사실상 앱 그 자체를 의미한다
- #### 우리가 작성한 커스텀 코드나 객체들, 그리고 우리가 앱의 기능이라고 생각하는 모든 것들은 다 UIApplication에 포함되어 있는 하위 객체이다
- #### 모바일 디바이스에 설치된 앱을 실행하면 초기 구동 과정을 거쳐 앱 프로세스가 메모리에 등록되는데, 이때의 앱 프로세스가 곧 UIApplication 객체라고 보아도 무방하다

### UIApplication 객체의 역활은 매우 다양하다
- #### 이벤트 루프나 다른 높은 수준의 앱 동작을 관리할 뿐만 아니라 푸시 알람과 같은 특수한 이벤트를 우리가 정의한 커스텀 객체인 델리게이트에게 알려주기도 한다
- #### 이 클래스를 특별한 일이 있거나 중대한 목적이 있는 경우가 아니라면 서브 클래싱 없이 그대로 사용한다
    - #### 굳이 서브 클래싱할 필요도 없고, 하기도 어렵기 때문이다

### 그런데 달리 생각해보면 UIApplication 객체를 서브 클래싱하지 않고 그대로 사용하는 것에는 한계가 있다
- #### 우리의 의도와 목적에 맞게 특별히 처리해야 할 것도 있을 수 있기 때문이다
- #### 그래서 UIApplication 객체는 AppDelegate라는 대리 객체를 내세우고 커스텀 코드를 처리할 수 있도록 약간의 권한을 부여한다

### AppDelegate는 UIApplication 으로부터 위임받은 일부 권한을 이용하여 커스텀 코드와 상호작용하는 역활을 담당
- #### 이를 통해 우리가 필요한 코드를 구현할 수 있도록 도와준다

### 이 관계가 쉽게 이해되지 않는다면 기업의 회장과 비서실의 관계를 떠올려 보면 된다
- #### 보통 회장은 회사의 아주 중요하고 핵심적인 일을 처리한다
    - #### 즉 회사의 신사업 구상이나 방향성 , 대외 업무 등을 담당하는 것이다
    - #### 하지만 사업을 구상하기 위해서는 자료 조사도 필요하고, 구상된 사업을 실제로 진척시키려면 현업에서 처리해야 할 일들이 많다
        - #### 이런 것들을 회장님이 직접 하지는 않는다
            - #### 대부분의 경우 큰 방향성에 대한 오더를 비서진에게 내린다
            - #### 이에 대한 권한을 위임받은 비서진은 이를 바탕으로 현업 부서에게 업무를 지시하고, 각 현업 부서는 이를 바탕으로 세부적인 업무를 짜서 진행하게 된다

### 여기서 회장은 UIApplication 객체이고, 비서실은 AppDelegate 객체라고 생각하면 된다
- #### 쉽게 말해 UIApplication 객체는 앱이 해야 할 아주 중요하고 핵심적인 일
    - #### 즉 앱의 생명 주기 관리나 이벤트 처리와 같은 것들을 담당
- #### AppDelegate는 커스텀 코드를 처리하는 비서의 성격을 띠는 것이다

### p 74 3번째 문단부터